Lien de téléchargement des données: https://cvml.ista.ac.at/AwA2/        

13GB file : https://cvml.ista.ac.at/AwA2/AwA2-data.zip

In [1]:
import sys
import cv2 # Pour utiliser open_cv, il faut la version de python est 3.7
import os
import csv

import numpy as np 
import pandas as pd 
import math

import torch 
from torch.utils.data import Dataset, DataLoader
import torchvision 
from torchvision.io import read_image
import torchvision.datasets as datasets
import torchvision.transforms as transforms

np.random.seed(0)

In [2]:
# Constant. Should be the path to the folder named JPEGImages, containing the 33K images in its subfolders.
DATA_FOLDER_PATH = "E:\\3710datas\\Animals_with_Attributes2\\"
JPEGIMAGES_FOLDER_PATH = "E:\\3710datas\\Animals_with_Attributes2\\JPEGImages\\"

# Note : Some labels have a low number of images. 

## Possible solutions to explore : 
    Data augmentation : creating new training data by applying random transformations to existing images, such as rotating, cropping, or flipping them.

In [5]:
def find_num_images_per_label(img_dir = JPEGIMAGES_FOLDER_PATH) -> tuple[dict,dict]: 
    """ 
    USEFUL FOR SAMPLING.
    Return a dict with keys as the 50 labels, and values being the number of images in each subdirectory corresponding to label
    and a second dict with the relative numbers (proportion) for every label compared to the total number of images (useful for sampling)"""
    labels_dirs = os.listdir(img_dir)
    num_images_per_label = dict.fromkeys(labels_dirs)
    proportions_images_per_label = dict.fromkeys(labels_dirs)
    total_num_images = 0

    # Update absolute number of images per label
    for i, label in enumerate(labels_dirs) : 
        specific_label_path = os.path.join(img_dir, labels_dirs[i])
        num_images_label = len(os.listdir(specific_label_path))
        total_num_images += num_images_label
        num_images_per_label[label] = num_images_label

    # Update relative number of images per label (proportion)
    for i, label in enumerate(labels_dirs) : 
        num_images_label = num_images_per_label[label]
        proportion_label = round(num_images_label / total_num_images, 4)
        proportions_images_per_label[label] = proportion_label

    return num_images_per_label, proportions_images_per_label

num_images_per_label, proportions_images_per_label = find_num_images_per_label()
print(num_images_per_label)
print(proportions_images_per_label)

{'antelope': 1046, 'bat': 383, 'beaver': 193, 'blue+whale': 174, 'bobcat': 630, 'buffalo': 895, 'chihuahua': 567, 'chimpanzee': 728, 'collie': 1028, 'cow': 1338, 'dalmatian': 549, 'deer': 1344, 'dolphin': 946, 'elephant': 1038, 'fox': 664, 'german+shepherd': 1033, 'giant+panda': 874, 'giraffe': 1202, 'gorilla': 872, 'grizzly+bear': 852, 'hamster': 779, 'hippopotamus': 684, 'horse': 1645, 'humpback+whale': 709, 'killer+whale': 291, 'leopard': 720, 'lion': 1019, 'mole': 100, 'moose': 704, 'mouse': 185, 'otter': 758, 'ox': 728, 'persian+cat': 747, 'pig': 713, 'polar+bear': 868, 'rabbit': 1088, 'raccoon': 512, 'rat': 310, 'rhinoceros': 696, 'seal': 988, 'sheep': 1420, 'siamese+cat': 500, 'skunk': 188, 'spider+monkey': 291, 'squirrel': 1200, 'tiger': 877, 'walrus': 215, 'weasel': 272, 'wolf': 589, 'zebra': 1170}
{'antelope': 0.028, 'bat': 0.0103, 'beaver': 0.0052, 'blue+whale': 0.0047, 'bobcat': 0.0169, 'buffalo': 0.024, 'chihuahua': 0.0152, 'chimpanzee': 0.0195, 'collie': 0.0275, 'cow': 0.

In [6]:
ANNOTATIONS_FILENAME = 'annotations.csv'

def create_annotations_csv_file(annotations_filename = ANNOTATIONS_FILENAME, img_dir = JPEGIMAGES_FOLDER_PATH): 
    """ 
    Create a csv annotations_file, annotations.csv, with two columns, in the format : 
                        path/to/image, label
    
    The annotation csv is necessary for DataLoader.
    """
    
    labels_dirs:list = os.listdir(img_dir)
   
    if os.path.exists(annotations_filename):
        os.remove(annotations_filename)
        print(f'Deleted existent {ANNOTATIONS_FILENAME} file.\n ---------------------------')
    
    with open(annotations_filename, 'w', newline='') as file :
        writer = csv.writer(file, dialect='excel', delimiter=',')

        for i, label in enumerate(labels_dirs) : 

            specific_label_path = os.path.join(img_dir, label)
            images_names = os.listdir(specific_label_path)

            for j, image_name in enumerate(images_names):
                full_path_to_img= os.path.join(specific_label_path, image_name)
                full_path_to_img= os.path.join(label, image_name)

                row = [full_path_to_img, label]
                writer.writerow(row)

    print(f'Sucessfully created {ANNOTATIONS_FILENAME} file.')

#
create_annotations_csv_file()

Deleted existent annotations.csv file.
 ---------------------------
Sucessfully created annotations.csv file.


In [7]:
# labels_in_number = pd.read_csv(DATA_FOLDER_PATH+"classes.txt", delim_whitespace=True,header=None)
labels_dict = {}
with open(DATA_FOLDER_PATH+"classes.txt") as f:
    for line in f:
        # print(line.split())
        (key,val) = line.split()
        labels_dict[val] = int(key)-1
print(labels_dict)

{'antelope': 0, 'grizzly+bear': 1, 'killer+whale': 2, 'beaver': 3, 'dalmatian': 4, 'persian+cat': 5, 'horse': 6, 'german+shepherd': 7, 'blue+whale': 8, 'siamese+cat': 9, 'skunk': 10, 'mole': 11, 'tiger': 12, 'hippopotamus': 13, 'leopard': 14, 'moose': 15, 'spider+monkey': 16, 'humpback+whale': 17, 'elephant': 18, 'gorilla': 19, 'ox': 20, 'fox': 21, 'sheep': 22, 'seal': 23, 'chimpanzee': 24, 'hamster': 25, 'squirrel': 26, 'rhinoceros': 27, 'rabbit': 28, 'bat': 29, 'giraffe': 30, 'wolf': 31, 'chihuahua': 32, 'rat': 33, 'weasel': 34, 'otter': 35, 'buffalo': 36, 'zebra': 37, 'giant+panda': 38, 'deer': 39, 'bobcat': 40, 'pig': 41, 'lion': 42, 'mouse': 43, 'polar+bear': 44, 'collie': 45, 'walrus': 46, 'raccoon': 47, 'cow': 48, 'dolphin': 49}


In [8]:
class AWA2Dataset(Dataset): # Dataset class to serve as input for the DataLoader.
    """ 
    Dataset class to serve as input for the DataLoader.
    Implements all the required methods and more. 
    """

    def __init__(self, annotations_file=ANNOTATIONS_FILENAME, img_dir=JPEGIMAGES_FOLDER_PATH, 
                transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

        numbers_infos_dicts: tuple[dict,dict] = find_num_images_per_label(img_dir=JPEGIMAGES_FOLDER_PATH)
        self.num_images_per_label = numbers_infos_dicts[0]
        self.proportions_images_per_label = numbers_infos_dicts[1]

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        # img_path = self.img_labels.iloc[idx, 0]
        key = self.img_labels.iloc[idx, 1]

        # Mapping the labels from string to tensor
        label = labels_dict[key]

        image = read_image(img_path)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [9]:
dataset = AWA2Dataset()
## TODO : Change transforms. Currently this is not useful.
dataset.transform = transforms.Compose([
                    transforms.ToPILImage(),
                    transforms.Resize(256),
                    transforms.CenterCrop(224),
                    transforms.Grayscale(num_output_channels=3),
                    transforms.ToTensor(), # Already a tensor as implemented in Dataset class with the reaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
                    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                ])

train_size =  int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size,test_size])

In [10]:
# Experiment with DataLoader. Everything works good
dataloader = DataLoader(dataset = dataset, batch_size=4, shuffle=True)
# dataiter = iter(dataloader)
# data = next(dataiter)

# images, labels = data 
# print(labels, images.shape)

In [15]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(64 * 56 * 56, 512)
        self.fc2 = nn.Linear(512, 50) 

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x.float())))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 56 * 56)
        x = self.dropout(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle= True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle= True)

model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

num_epochs = 2
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_dataloader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_dataloader)}], Loss: {loss.item():.4f}")


Epoch [1/2], Step [1/7464], Loss: 3.9169
Epoch [1/2], Step [2/7464], Loss: 3.8877
Epoch [1/2], Step [3/7464], Loss: 3.9056
Epoch [1/2], Step [4/7464], Loss: 3.9185
Epoch [1/2], Step [5/7464], Loss: 3.8029
Epoch [1/2], Step [6/7464], Loss: 3.9254
Epoch [1/2], Step [7/7464], Loss: 3.8979
Epoch [1/2], Step [8/7464], Loss: 3.9225
Epoch [1/2], Step [9/7464], Loss: 3.8364
Epoch [1/2], Step [10/7464], Loss: 3.8989
Epoch [1/2], Step [11/7464], Loss: 3.8267
Epoch [1/2], Step [12/7464], Loss: 3.7474
Epoch [1/2], Step [13/7464], Loss: 4.1727
Epoch [1/2], Step [14/7464], Loss: 3.8569
Epoch [1/2], Step [15/7464], Loss: 3.8428
Epoch [1/2], Step [16/7464], Loss: 3.4560
Epoch [1/2], Step [17/7464], Loss: 4.0778
Epoch [1/2], Step [18/7464], Loss: 3.6479
Epoch [1/2], Step [19/7464], Loss: 3.9229
Epoch [1/2], Step [20/7464], Loss: 4.2268
Epoch [1/2], Step [21/7464], Loss: 3.6706
Epoch [1/2], Step [22/7464], Loss: 4.0509
Epoch [1/2], Step [23/7464], Loss: 3.6729
Epoch [1/2], Step [24/7464], Loss: 3.9700
E

KeyboardInterrupt: 

In [16]:

model.eval() 

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dataloader:

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        print('Accuracy of the model on the test images: {:.2f}%'.format(100 * correct / total))

Accuracy of the model on the test images: 25.00%
Accuracy of the model on the test images: 12.50%
Accuracy of the model on the test images: 8.33%
Accuracy of the model on the test images: 12.50%
Accuracy of the model on the test images: 20.00%
Accuracy of the model on the test images: 16.67%
Accuracy of the model on the test images: 17.86%
Accuracy of the model on the test images: 18.75%
Accuracy of the model on the test images: 16.67%
Accuracy of the model on the test images: 15.00%
Accuracy of the model on the test images: 15.91%
Accuracy of the model on the test images: 14.58%
Accuracy of the model on the test images: 15.38%
Accuracy of the model on the test images: 14.29%
Accuracy of the model on the test images: 15.00%
Accuracy of the model on the test images: 15.62%
Accuracy of the model on the test images: 16.18%
Accuracy of the model on the test images: 16.67%
Accuracy of the model on the test images: 15.79%
Accuracy of the model on the test images: 16.25%
Accuracy of the model